In [1]:
import gpt as g

## Lecture 2: Linear algebra with lattices

We start by creating a $4^4$ double precision grid.

In [2]:
grid = g.grid([4, 4, 4, 4], g.double)

Next, we create a parallel pseudorandom number generator and two color vectors from a complex normal distribution.

In [3]:
rng = g.random("seed for rng")
v1 = g.vcolor(grid)
v2 = g.vcolor(grid)
rng.cnormal([v1,v2]);

GPT :       0.401606 s : Initializing gpt.random(seed for rng,vectorized_ranlux24_389_64) took 0.000459671 s


We inspect a one-dimensional slice of $v_1$.

In [4]:
v1[0,0,0,:]

array([[-0.58527248-0.85360387j, -1.580872  -1.50785674j,
        -1.04090035+0.40186692j],
       [-0.23550862-0.90891595j,  0.73888871+0.21903038j,
         1.10074976+1.52119033j],
       [ 0.79255516+0.64832885j, -0.22812566+0.54092838j,
         1.17416062-0.72316771j],
       [-0.81076797+0.6141707j ,  0.6581139 -0.02166552j,
         0.17446881+0.19891321j]])

Let us now take a linear combination of $\frac12 v_1 + 3 v_2$.  Note that there is a difference between abstract expressions and evaluating them to lattice fields.

In [5]:
expr = 1./2. * v1 + 3. * v2

g.message("This is an expression:", expr)

result = g.eval(expr)

g.message("And this is the coordinate (0,0,0,0) of the evaluated lattice object:", result[0,0,0,0])

GPT :       0.439875 s : This is an expression:  + ((0.5+0j))*lattice(ot_vector_color(3),double) + ((3+0j))*lattice(ot_vector_color(3),double)
GPT :       0.445578 s : And this is the coordinate (0,0,0,0) of the evaluated lattice object: tensor([ 0.09215363+4.58520759j  0.53058119-4.0223991j  -0.79159505+2.5800456j ],ot_vector_color(3))


Since the "=" operator in Python cannot be overloaded and always assigns the Python object on the right side to the symbol to left, GPT uses the operator "@=" to assign the result of an expression to a lattice field.

In [6]:
result2 = g.lattice(v1) # creates a lattice of same type than v1 without initializing the values
result2 @= expr

g.message("Difference between g.eval(...) and @= operation:", g.norm2(result2 - result))

GPT :       0.456365 s : Difference between g.eval(...) and @= operation: 0.0


It is also useful to know of the increment operator:

In [7]:
result += v1

Next, let us construct site-local inner and outer products from our vectors.

In [8]:
local_inner_product = g.complex(grid)
local_inner_product @= g.adj(v1) * v2

local_outer_product = g.mcolor(grid)
local_outer_product @= v1 * g.adj(v2)

g.message("Origin of inner_product field:", local_inner_product[0,0,0,0])

g.message("Origin of outer_product field:", local_outer_product[0,0,0,0])


GPT :       0.479349 s : Origin of inner_product field: (-0.14170995506280581+0.7288503091897771j)
GPT :       0.482812 s : Origin of outer_product field: tensor([[-1.50115922+0.86831104j  0.67227476-1.01352379j -0.6240419 +0.54129306j]
                       :  [-2.72189862+2.44771251j  0.94667551-2.38631285j -1.05290498+1.38997313j]
                       :  [ 0.53787765+1.79054559j -0.89617918-0.95709307j  0.41277376+0.7891515j ]],ot_matrix_su_n_fundamental_group(3))


In [9]:
v12 = g.inner_product(v1, v2)
v11 = g.inner_product(v1, v1)
n1 = g.norm2(v1)

g.message("v1 . v2", v12)
g.message("sum(local_inner_product)", g.sum(local_inner_product))

g.message("v1 . v1", v11)
g.message("norm2(v1)", n1)


GPT :       0.496987 s : v1 . v2 (-44.827357722843956-52.851620669240525j)
GPT :       0.499137 s : sum(local_inner_product) (-44.82735772284396-52.851620669240525j)
GPT :       0.500471 s : v1 . v1 (1592.0538142528967-2.14659970599242e-15j)
GPT :       0.501776 s : norm2(v1) 1592.0538142528967


We move on to SU(3) matrices now.  g.mcolor is short-hand for the QCD gauge group, but GPT has general SU(N) and U(1) implemented.  We initialize a SU(3) matrix with a random near-unit element and then compute its site-wise determinant and inverse and check the results.

In [10]:
V = g.mcolor(grid)
rng.normal_element(V, scale=0.01)

g.message("Origin of V:", V[0,0,0,0])

GPT :       0.526540 s : Origin of V: tensor([[ 0.99992903-0.00286107j -0.00411134-0.01041353j  0.00277436+0.00083957j]
                       :  [ 0.00410881-0.01043508j  0.99991181+0.00142932j -0.00571668+0.00398393j]
                       :  [-0.00270793+0.00080031j  0.00572503+0.00402551j  0.9999705 +0.00143171j]],ot_matrix_su_n_fundamental_group(3))


In [11]:
det_V = g.matrix.det(V)

g.message("Slice of local determinant:", det_V[0,0,0,:])

GPT :       0.562489 s : Slice of local determinant: [[1.-3.14017533e-20j]
                       :  [1.-6.78225009e-19j]
                       :  [1.-2.12880792e-20j]
                       :  [1.+1.37204908e-18j]]


In [12]:
inv_V = g.matrix.inv(V)

g.message("Difference between matrix inverse and adjoint for unitary matrix:", g.norm2(inv_V - g.adj(V)))

GPT :       0.575219 s : Difference between matrix inverse and adjoint for unitary matrix: 1.849405542690569e-29


We can also check that the logarithm of the matrix is anti-Hermitian.

In [13]:
logV = g.matrix.log(V)
g.norm2(logV + g.adj(logV))

2.1647695871037737e-29

There are also component-wise operations, see, e.g., the cosine:

In [14]:
g.component.cos(V)[0,0,0,0]

tensor([[0.54036423+2.40739705e-03j 1.00004577-4.28141474e-05j
  0.9999965 -2.32927134e-06j]
 [1.000046  +4.28763992e-05j 0.54037706-1.20266577e-03j
  0.9999916 +2.27747790e-05j]
 [0.99999665+2.16718170e-06j 0.99999171-2.30460831e-05j
  0.54032768-1.20471735e-03j]],ot_matrix_su_n_fundamental_group(3))